# Import

In [1]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
from torch.optim import AdamW

batch_size = 4

# eval summarize, -1-, summary

## Create test Dataset and Dataloader

In [2]:
tokenizer = T5Tokenizer.from_pretrained("kiri-ai/t5-base-qa-summary-emotion")

class JsonlDataset(Dataset):
    def __init__(self, filename, is_test=False):
        self.data = [json.loads(line) for line in open(filename, 'r', encoding='utf-8')]
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def getMerged(self, question, options, article):
        question = question.replace('@placeholder', '_1_')
        options_str = ', '.join(f'{options[i]}' for i in range(5))     
        return f'summarize: Summary<{question}>\nGiven Options<{options_str}>\nAnd Article<{article}>'
    
    def getLabel(self, question, options, label):
        answer = f"{options[label]}"
        return question.replace('@placeholder', answer)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        article = item['article']
        question = item['question']
        options = [item[f'option_{i}'] for i in range(5)]
        
        merged = self.getMerged(question, options, article)
        
        in_tokenize = tokenizer(
            merged,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        input_ids = in_tokenize['input_ids'].flatten()
        attention_mask = in_tokenize["attention_mask"].flatten()
        
        if self.is_test:
            return {
                'article': article,
                'question': question,
                'options': options,
                'input_ids': input_ids,
                'attention_mask': attention_mask,
                'merged': merged
            }
        else:
            out_tokenize = tokenizer(
                self.getLabel(question, options, item['label']),
                add_special_tokens=True,
                max_length=150,
                return_tensors="pt",
                padding='max_length'
            )
            
            label = out_tokenize['input_ids'].flatten()
            
            return {
                'article': article,
                'question': question,
                'options': options,
                'label': label,
                'input_ids': input_ids,
                'attention_mask': attention_mask, 
                'label': label,
                'merged': merged
            }

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
task1_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [4]:
def eval(model, test_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model = model.to(device)
    model.eval()
    
    total = 0
    correct = 0
    for i, batch in enumerate(test_loader):
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            label = batch['label'].to(device)
            
            tokens = model.generate(
                input_ids=input_ids, 
                attention_mask=attention_mask,
                max_length=150
            )

            gt = [
                tokenizer.decode(
                    label,
                    skip_special_tokens=True,
                ) for label in label
            ]

            preds = [tokenizer.decode(token, skip_special_tokens=True) for token in tokens]
            
            if i == 0:
                for i, token in enumerate(tokens):
                    print(gt[i])
                    print(preds[i])
                    print('\n')

            total += len(gt)
            correct += sum(gt[i] == preds[i] for i in range(len(preds)))

    print(correct / total)

## Evaluation Task1, Cross1, Task2, Cross2

In [5]:
config = T5Config.from_pretrained("kiri-ai/t5-base-qa-summary-emotion")

model = T5ForConditionalGeneration.from_pretrained('../input/semevalmodelt5/task1_1_summary_best.bin', config=config)
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('../input/semevalmodelt5/task2_1_summary_best.bin', config=config)
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
Bollywood star Salman Khan's fans are jubilant after a Mumbai court suspended his five - year jail sentence over a 2002 hit - and - run case, but others feel justice has been denied.
Bollywood star Salman Khan's fans are jubilant after a Mumbai court suspended his five - year jail sentence over a 2002 hit - and - run case, but others feel justice has been denied.


The only Russian due to compete in the athletics at Rio 2016 has been cleared to contest the women's long jump after successfully appealing against a ban.
The only Russian due to compete in the athletics at Rio 2016 has been cleared to contest the women's long jump after successfully appealing against a ban.


Kirsty Gilmour says UK Sport's decision to withdraw its funding for badminton over the next Olympic cycle will cost her nearly £ 30,000 a year.
Kirsty Gilmour says UK Sport's decision to withdraw its funding for badminton over the next Olympic cycle will cost her nearly £ 30,000 a year.


Margate's troubled Dre

## Evaluation without fine tuning

In [6]:
model = model.from_pretrained('kiri-ai/t5-base-qa-summary-emotion')
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('kiri-ai/t5-base-qa-summary-emotion')
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
Wales have moved up one place in the latest Fifa football rankings to a new high of ninth.
Wales have moved up one place in the _1_ Fifa football rankings to a new high of ninth. The Netherlands have dropped down to 12th, England have climbed a place to eighth. Argentina remain in the number one spot, with Belgium back up to second and Germany in third.


All the latest team news and stats for Saturday's Premier League, Championship and Scottish Premiership fixtures.
Crystal Palace host Sunderland Everton at St James' Park on Saturday. Sunderland host Bournemouth Hull City at St James' Park. Southampton host West Ham United at St James' Park. Barnsley host Fulham Blackburn Rovers at Preston North End. Reading host Reading United


The UK will remain a major contributor to European security, Barack Obama has said, as Nato and EU nations signed a deal focusing on the perceived threat from Russia.
UK PM David Cameron says UK will not "turn its back" on European defence. Nato and E

# cloze, @placehodler, i-option[i]

In [7]:
class JsonlDataset2(JsonlDataset):
    def __init__(self, filename, is_test=False):
        super().__init__(filename, is_test)

    def getMerged(self, question, options, article):
        options_str = ', '.join(f'{i}-{options[i]}' for i in range(5))     
        return f'cloze: Summary<{question}>\nGiven Options<{options_str}>\nAnd Article<{article}>'
        
    def getLabel(self, question, options, label):
        label = f"{label}-{options[label]}"
        return label

In [8]:
task1_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_1_Imperceptibility.jsonl')
task1_test_loader = DataLoader(task1_test_dataset, batch_size=batch_size, shuffle=True)


task2_test_dataset = JsonlDataset2('../input/semevaldataset/trail_data/Task_2_Nonspecificity.jsonl')
task2_test_loader = DataLoader(task2_test_dataset, batch_size=batch_size, shuffle=True)

In [9]:
model = model.from_pretrained('../input/semevalmodelt5/task1_cloze_option_best.bin', config=config)
print("Task1: ")
eval(model, task1_test_loader)
print("Task1 Cross: ")
eval(model, task2_test_loader)

model = model.from_pretrained('../input/semevalmodelt5/task2_cloze_option_best.bin', config=config)
print("Task2: ")
eval(model, task2_test_loader)
print("Task2 Cross: ")
eval(model, task1_test_loader)

Task1: 
4-prove
3-inspire


1-personal
4-controversial


4-statutory
0-control


3-uncomfortable
3-uncomfortable


0.238
Task1 Cross: 
1-collapse
1-collapse


2-document
0-suggests


3-collapse
1-record


2-event
0-direction


0.212
Task2: 
3-activities
2-prepares


4-head
2-legality


1-collapse
2-situation


0-style
1-controlled


0.17
Task2 Cross: 
3-effective
0-suffering


1-rare
1-rare


4-around
1-almost


0-role
1-inspiration


0.223
